In [241]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [242]:
csv_file_path = '/home/lucas/UFOP/ple_2020/analise_midias_sociais/final-work/data/all_pp_posts_bolsonaro_posts.csv'
df_candidate = pd.read_csv(csv_file_path,encoding = 'utf-8')

In [243]:
df_candidate.head(15)

,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
0,2020-03-22T18:47:55+0000,211857482296579_1826486634166981,agor videoconferenc president jair bolsonar pr...,- agora por videoconferencia o presidente jair...,23151,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,129465,147,318,21,112,10345,True
1,2020-03-22T16:30:39+0000,211857482296579_1826339070848404,inscrico seleca profission atuaca program medi...,- as inscricoes para a selecao de 5.800 profis...,5703,mobile_status_update,None,52238,128,61,5,43,2514,True
2,2020-03-22T13:37:26+0000,211857482296579_1826130007535977,ministeri saud ministr mandett reunia virtual ...,- estamos no ministerio da saude com o ministr...,11818,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,97986,73,141,16,65,6292,True
3,2020-03-22T11:07:01+0000,211857482296579_1825937284221916,govern federal distribu milho test rap covid t...,- o governo federal distribui milhoes de teste...,39284,mobile_status_update,None,165539,88,393,20,68,16747,True
4,2020-03-22T02:06:48+0000,211857482296579_1825422034273441,cham precauca medic barat cas venh ser comprov...,- isso se chama precaucao . o medicamento e ba...,27236,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9052732...,162513,968,318,88,509,20723,True
5,2020-03-21T22:22:48+0000,211857482296579_1825061764309468,reconhec seriedad moment temor muit brasileir ...,- reconheco a seriedade do momento e o temor d...,40794,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9065012...,305358,918,478,266,1267,48834,True
6,2020-03-21T18:36:01+0000,211857482296579_1824728371009474,hospital albert einstein possivel cur pacient ...,- hospital albert einstein e a possivel cura d...,201803,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/75...,215237,480,815,60,431,34278,True
7,2020-03-21T11:33:30+0000,211857482296579_1824261207722857,dois ded pros ratinh assist link youtub,- dois dedos de prosa com ratinho . - assista ...,33971,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/83...,106117,336,317,113,624,15368,True
8,2020-03-21T02:37:00+0000,211857482296579_1823872341095077,assin med provisor deix clar ser competenc fed...,- assinei medida provisoria deixando claro ser...,9174,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/s720x72...,112873,181,200,106,1121,7582,True
9,2020-03-21T00:16:41+0000,211857482296579_1823743497774628,via eduard bolsonar,via eduardo bolsonaro,8150,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p600x60...,79762,435,111,13,160,6480,True


In [244]:
n_total_posts = len(df_candidate)
print('número total de posts: {0}'.format(str(n_total_posts)))

número total de posts: 3646


In [245]:
n_posts_with_textual_messages = df_candidate[df_candidate.has_textual_message == True].shape[0]
n_posts_without_textual_messages = df_candidate[df_candidate.has_textual_message == False].shape[0]

print('número de posts com mensagens textuais: {0}'.format(str(n_posts_with_textual_messages)))
print('número de posts sem mensagens textuais: {0}'.format(str(n_posts_without_textual_messages)))

número de posts com mensagens textuais: 3394
número de posts sem mensagens textuais: 252


In [180]:
# Devemos remover esses posts sem menssagem antes de continuar a nossa análise?
# renomear depois o df caso sim

#filter_mask  = df_candidate['has_textual_message'] == True
#df_candidate = df_candidate[filter_mask]
#print('número de posts do df desconsiderando os posts sem menssagens textuais: {0}'.format(str(len(df_candidate))))

número de posts do df desconsiderando os posts sem menssagens textuais: 3394


In [246]:
# separando o df de treinamento (30%) e o df de testes(70%) de maneira 'embaralhada'
df_training, df_test = train_test_split(df_candidate,train_size=0.7,test_size=0.3,shuffle=True)

In [247]:
print('número de posts do conjunto de treinamento (70%): {0}'.format(str(len(df_training))))
df_training.head(5)

número de posts do conjunto de treinamento (70%): 2552


,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
2785,2016-03-14T00:55:23+0000,211857482296579_600966793385644,direit bolsonar chut esquerd brasil via carl b...,- '' com a direita bolsonaro chuta a esquerda ...,12598,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/12...,48378,1427,75,6,18,1030,True
1878,2017-09-18T15:30:18+0000,211857482296579_917267808422206,entrev radi itatia,- entrevista radio itatiaia .,9070,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/21...,18614,58,122,4,14,1172,True
3202,2014-12-05T20:37:56+0000,211857482296579_410377239111268,hoj encontr forc espec comand paraqued forc ar...,hoje encontro de forcas especiais comandos e p...,3417,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/s720x72...,43448,0,1,0,1,5,True
1288,2018-07-31T12:39:53+0000,211857482296579_1139547722860879,,,3063,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/37...,22289,133,75,4,37,3847,False
1413,2018-05-24T11:37:45+0000,211857482296579_1073544709461181,vereador otton paul riorj bolsonar ditador,- vereador ottoni de paula rio/rj : `` bolsona...,27665,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/29...,37083,583,193,10,60,2151,True


In [248]:
print('número de posts do conjunto de teste (30%): {0}'.format(str(len(df_test))))
df_test.head(5)

número de posts do conjunto de teste (30%): 1094


,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
2769,2016-03-17T22:20:01+0000,211857482296579_602822096533447,brasil agor hmin congress nacional,brasilia agora 19h15min . congresso nacional .,8770,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/12...,43741,173,72,6,21,686,True
1384,2018-06-01T21:29:24+0000,211857482296579_1079011432247842,moura bolsonar melhor brasil,mourao : bolsonaro e o melhor para o brasil !,3755,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/32...,9386,23,53,1,11,526,True
2447,2016-09-15T16:42:44+0000,211857482296579_689810714501251,edenilson garc candidat barr mans rj grat apoi...,edenilson garcia - 20.120 - e nosso candidato ...,200,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/14...,3069,9,1,2,1,42,True
3057,2015-06-19T22:32:26+0000,211857482296579_500779356737722,sargent aeronaut formatur guaratinguet sp form...,sargentos da aeronautica - formatura - guarati...,776,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/s720x72...,21039,0,1,0,1,11,True
1223,2018-08-28T20:03:27+0000,211857482296579_2131071940545169,port alegr agostoquart feir pous h aeroport sa...,- porto alegre/29 de agosto/quarta- feira . - ...,8585,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/38...,33835,100,163,6,67,2350,True


In [271]:
#============================================================================================
# realizando o tf idf no df['pre_processed_message'] de treinamento
#============================================================================================

# instanciando o objeto para fazer o tf-idf
tfidf_vectorizer = TfidfVectorizer(analyzer='word')

# converte os documentos em matrizes
tfidf_matrix = tfidf_vectorizer.fit_transform(df_training['pre_processed_message'])

# obtendo as palavras (tokens)
tfidf_tokens = tfidf_vectorizer.get_feature_names()

# gerando o tf-idf em um df
# colunas são as palavras processadas
# linhas são os documentos (post 1, post 2,post 3,post 4)
df_tfidf_df = pd.DataFrame(data = tfidf_matrix.toarray(),columns = tfidf_tokens)


print("\nTF-IDF\n")
df_tfidf_df.head(10)


TF-IDF



,abaix,abandon,abastec,abat,abav,abcz,abdelmassih,abenco,aberraco,abert,...,zarattin,ze,zec,zelos,zer,zl,zoeir,zon,zueir,zuer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [270]:
# Aplicando uma máscara para identificar o tf-idf de certas palavras

filter_mask  = df_tfidfvect['zarattin'] != 0.0
df_tfidf_token_mask = df_tfidfvect[filter_mask]

print(df_tfidf_token_mask)

      abaix  abandon  abastec  abat  abav  abcz  abdelmassih  abenco  \
463     0.0      0.0      0.0   0.0   0.0   0.0          0.0     0.0   
1342    0.0      0.0      0.0   0.0   0.0   0.0          0.0     0.0   

      aberraco  abert  ...  zarattin   ze  zec  zelos  zer   zl  zoeir  zon  \
463        0.0    0.0  ...  0.430778  0.0  0.0    0.0  0.0  0.0    0.0  0.0   
1342       0.0    0.0  ...  0.356042  0.0  0.0    0.0  0.0  0.0    0.0  0.0   

      zueir  zuer  
463     0.0   0.0  
1342    0.0   0.0  

[2 rows x 6535 columns]


In [ ]:
#============================================================================================
# Implementacao do lda
#============================================================================================



In [224]:
cv = CountVectorizer(max_df = 0.8, min_df = 2)
df = cv.fit_transform(df_candidate['pre_processed_message'])

In [20]:
model_lda = LatentDirichletAllocation(n_components = 10)
model_lda.fit(df)

LatentDirichletAllocation()

In [14]:
for index, topic in enumerate(model_lda.components_):
    print(f'Top 10 words for #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for #0
['brasil', 'milit', 'alun', 'federal', 'public', 'cas', 'ensin', 'program', 'educaca', 'escol']


Top 10 words for #1
['pod', 'pov', 'pt', 'vid', 'polit', 'tud', 'acim', 'tod', 'nao', 'brasil']


Top 10 words for #2
['cam', 'deput', 'glob', 'direit', 'sobr', 'jornal', 'brasil', 'verdad', 'bolsonar', 'nao']


Top 10 words for #3
['part', 'hoj', 'milit', 'tod', 'janeir', 'grand', 'abrac', 'rio', 'brasil', 'obrig']


Top 10 words for #4
['sao', 'seguranc', 'econom', 'nao', 'brasil', 'med', 'public', 'ministr', 'govern', 'ministeri']


Top 10 words for #5
['infraestrutur', 'saud', 'nov', 'ministr', 'milho', 'estad', 'econom', 'govern', 'brasil', 'ministeri']


Top 10 words for #6
['tod', 'ser', 'dilm', 'president', 'eduard', 'sao', 'jair', 'deput', 'nao', 'bolsonar']


Top 10 words for #7
['agor', 'flavi', 'sobr', 'eduard', 'brasil', 'tod', 'youtub', 'link', 'jair', 'bolsonar']


Top 10 words for #8
['jair', 'dilm', 'sobr', 'contr', 'deput', 'lei', 'brasil', 'nao', 'pt'

In [19]:
feature_names = cv.get_feature_names()
number_top_words = 10
for topic_idx, topic in enumerate(model_lda.components_):
    message = "Topic {0}: ".format(str(topic_idx))
    message += " ".join([feature_names[i]
                         for i in topic.argsort()[:-number_top_words - 1:-1]])
    print(message)
print()

Topic 0: brasil todos nao acima tudo governo deus sao pais pt
Topic 1: governo ministerio brasil programa bilhoes milhoes anos federal ano economia
Topic 2: sao pt nao paulo verdade sobre jornal comissao camara so
Topic 3: bolsonaro rio nao carlos janeiro brasil ministerio dilma obrigado flavio
Topic 4: bolsonaro jair youtube link brasil sobre eduardo entrevista obrigado presidente
Topic 5: bolsonaro federal nao eduardo lei brasil deputado sp canal sao
Topic 6: todos voto nao impresso bolsonaro hoje abraco brasil povo obrigado
Topic 7: nao bolsonaro educacao direitos militar humanos brasil sobre escolas rio
Topic 8: nao todos brasil presidente ser ministro bolsonaro boa apoio estado
Topic 9: obrigado nao pt contra brasil publica psol crime grande ministerio

